**Lab 9**

Name: 
Date: 

**As you work through the code below, respond to all in-line comments and questions.**

# Autoencoder

First, let's try building a simple autoencoder for the MNIST data.  This lab is loosely modeled after [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html).

In [ ]:
from tensorflow.keras.datasets import mnist

# Load in the MNIST data
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

N = int(.8 * train_X.shape[0])
val_X = train_X[N:].reshape(-1,784) / 255
val_Y = train_Y[N:]
train_X = train_X[:N].reshape(-1,784) / 255
train_Y = train_Y[:N]

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input
from keras import regularizers

print(train_X.shape)

input_img = Input(shape=(784,))
# Add a Dense layer with a L1 activity regularizer
encoded = Dense(64, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(train_X, train_X,
                epochs=25,
                batch_size=256,
                shuffle=True,
                validation_data=(val_X, val_X))


Now let's look at the enconding and reconstruction.

In [ ]:
# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

reconstructions = autoencoder.predict(val_X)
encodings = encoder.predict(val_X)

idx = 0

print("Encoded representation")
print(encodings[idx,:])

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Reconstruction")
image = reconstructions[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()




Go back and try to enforce a sparse encoding.  You may need to try to change the node type or number of nodes in the middle layer.  

>  *encoded = Dense(64, activation='relu',*
                *activity_regularizer=regularizers.l1(10e-5))(input_img)*

**1. What was the architecture of the model that you used for your sparse encoder?  How well do you think it performs?**

# Denoising Autoencoder

Let's add some noise to the input and see if we can do some reconstructions.

In [ ]:
# Reload the data but add some noise
from tensorflow.keras.datasets import mnist
import numpy as np

# Load in the MNIST data
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

N = int(.8 * train_X.shape[0])
val_X = train_X[N:].reshape(-1,784) / 255.
val_Y = train_Y[N:]
train_X = train_X[:N].reshape(-1,784) / 255.
train_Y = train_Y[:N]

noisy_train_X = train_X + 0.3 * np.random.normal(size=train_X.shape)
noisy_val_X = val_X + 0.3 * np.random.normal(loc=0.0, scale=1.0, size=val_X.shape)

noisy_train_X = np.clip(noisy_train_X, 0., 1.)
noisy_val_X = np.clip(noisy_val_X, 0., 1.)

# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

idx = 0

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Noisy image")
image = noisy_val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

Build, evaluate and visualize a denoising autoencoder.  

**2. How well does it do removing noise?  How well does it do reconstructing clean images?**

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input
from keras import regularizers

# Build, evaluate and visualize a denoising autoencoder

#input_img = Input(shape=(784,))

# ...


In [ ]:
# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

reconstructions = autoencoder.predict(val_X)
#reconstructions = autoencoder.predict(noisy_val_X)

idx = 89

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Noisy image")
image = noisy_val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Reconstruction")
image = reconstructions[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

# Spy Autoencoder

Consider the task of filling in a portion of the image that has been covered up.   

In [ ]:
# Reload the data but add some noise
from tensorflow.keras.datasets import mnist
import numpy as np
import random

# Load in the MNIST data
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

patch_size = 8

N = int(.8 * train_X.shape[0])
val_X = train_X[N:] / 255.
val_Y = train_Y[N:]
train_X = train_X[:N] / 255.
train_Y = train_Y[:N]

blotted_train_X = np.copy(train_X)
blotted_val_X = np.copy(val_X)

for digit in blotted_train_X:

    x_offset = random.randint(10, 18)
    y_offset = random.randint(10, 18)

    digit[x_offset:x_offset + patch_size, y_offset:y_offset + patch_size] = 1.

for digit in blotted_val_X:

    x_offset = random.randint(8, 20)
    y_offset = random.randint(8, 20)

    digit[x_offset:x_offset + patch_size, y_offset:y_offset + patch_size] = 1.

idx = 0

blotted_train_X = blotted_train_X.reshape((-1, 784))
blotted_val_X = blotted_val_X.reshape((-1, 784))
train_X = train_X.reshape((-1, 784))
val_X = val_X.reshape((-1, 784))

# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Blotted image")
image = blotted_val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

Build, evaluate and visualize an autoencoder to spy behind the plotted portion of the image.

**3. Does it make any difference if the square is filled with black instead of white?  How well can the model reconstruct the missing region.  Try with larger and smaller missing portions.** 

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input
from keras import regularizers

# Build, evaluate and visualize a denoising autoencoder

#input_img = Input(shape=(784,))

# ...

#autoencoder.fit(blotted_train_X, train_X,
                epochs=25,
                batch_size=256,
                shuffle=True,
                validation_data=(blotted_val_X, val_X))

In [ ]:
# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

reconstructions = autoencoder.predict(blotted_val_X)

idx = 89

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Noisy image")
image = blotted_val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Reconstruction")
image = reconstructions[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

# Convolutional Autoencoder

In [ ]:
from tensorflow.keras.datasets import mnist

# Load in the MNIST data
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

N = int(.8 * train_X.shape[0])
val_X = train_X[N:].reshape((-1,28,28,1)) / 255.
val_Y = train_Y[N:]
train_X = train_X[:N].reshape((-1,28,28,1)) / 255.
train_Y = train_Y[:N]



In [ ]:
# https://blog.keras.io/building-autoencoders-in-keras.html

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input, Conv2D, MaxPooling2D, UpSampling2D

input_img = Input(shape=(28, 28, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(train_X, train_X,
                epochs=25,
                batch_size=256,
                shuffle=True,
                validation_data=(val_X, val_X))

In [ ]:
# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

reconstructions = autoencoder.predict(val_X)

idx = 0

print("Original image")
image = val_X[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Reconstruction")
image = reconstructions[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()




# Derotational Convolutional Autoencoder

Let's use the convolutional autoencoder to "undo" rotations and shifts.  

**4. How well does the detransformational autoencoder do?  Do you think a non-convolutional autoencoder would be better suited for this task?  Why or why not?**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(
#        rotation_range=40,
#        width_shift_range=0.2,
#        height_shift_range=0.2,
#        shear_range=0.2,
#        zoom_range=0.2,
#        horizontal_flip=True,
#        fill_mode='nearest')

datagen = ImageDataGenerator(
        rotation_range=30)

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input, Conv2D, MaxPooling2D, UpSampling2D

#input_img = Input(shape=(28, 28))
input_img = Input(shape=(28, 28, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(datagen.flow(train_X, train_X, batch_size=32), steps_per_epoch=len(train_X) / 32, 
          epochs=10, validation_data=datagen.flow(val_X, val_X, batch_size=32))

In [ ]:
for transformed_X_batch, original_X_batch in datagen.flow(val_X, val_X, batch_size=32):

    loss_and_metrics = autoencoder.evaluate(transformed_X_batch, transformed_X_batch)
    print(loss_and_metrics)

    reconstructions = autoencoder.predict(transformed_X_batch)

    break


# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

idx = 0

print("Original image")
image = original_X_batch[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Transfomred image")
image = transformed_X_batch[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

print("Reconstruction")
image = reconstructions[idx,:]
plt.imshow(image.reshape(28,28),cmap='gray')
plt.show()

**5. Try some other transformations or combinations of transformations.  What did you try and how well did they do?**


**Submission:** Submit this notebook and a PDF of its generated output via Blackboard by the end of the day.  Submit the files individually.